In [ ]:
import pandas as pd
import polars as pl
import numpy as np
import time
import random
import copy
import rtsvg
rt = rtsvg.RACETrack()

from spreadlines_prototyping import spreadLines

df1 = pl.read_csv('../../data/2013_vast_challenge/mc3_netflow/nf/nf-chunk1.csv')
df2 = pl.read_csv('../../data/2013_vast_challenge/mc3_netflow/nf/nf-chunk2.csv')
df3 = pl.read_csv('../../data/2013_vast_challenge/mc3_netflow/nf/nf-chunk3.csv')
df  = pl.concat([df1, df2, df3])

df = rt.columnsAreTimestamps(df, 'parsedDate')
df = df.rename({'TimeSeconds':                '_del1_',
                'parsedDate':                 'timestamp',
                'dateTimeStr':                '_del2_',
                'ipLayerProtocol':            'pro',
                'ipLayerProtocolCode':        '_del3_',
                'firstSeenSrcIp':             'sip',
                'firstSeenDestIp':            'dip',
                'firstSeenSrcPort':           'spt',
                'firstSeenDestPort':          'dpt',
                'moreFragments':              '_del4_',
                'contFragments':              '_del5_',
                'durationSeconds':            'dur',
                'firstSeenSrcPayloadBytes':   '_del6_',
                'firstSeenDestPayloadBytes':  '_del7_',
                'firstSeenSrcTotalBytes':     'soct',
                'firstSeenDestTotalBytes':    'doct',
                'firstSeenSrcPacketCount':    'spkt',
                'firstSeenDestPacketCount':   'dpkt',
                'recordForceOut':             '_del8_'})
df = df.drop(['_del1_', '_del2_', '_del3_', '_del4_', '_del5_', '_del6_', '_del7_', '_del8_'])
#df = df.sample(100_000)

In [ ]:
# original impl:               about 14s per render ... total is about 43s
# w/out the alter concat step: about  9s per render ... total is about 28s
params = {'rt_self':rt, 'df':df, 'relationships':[('sip','dip')], 'node_color':'node', 'every':'1d', 'h':384}
sl0   =       spreadLines(node_focus='172.30.0.4',                                                           **params)
sl0_2 = rtsvg.spreadLines(node_focus='172.30.0.4',                                                           **params)
rt.tile([sl0, sl0_2], horz=False)

In [ ]:
sl1   =       spreadLines(node_focus='172.30.0.4', only_render_nodes=set(['10.156.165.212', '10.16.5.15',]), **params)
sl1_2 = rtsvg.spreadLines(node_focus='172.30.0.4', only_render_nodes=set(['10.156.165.212', '10.16.5.15',]), **params)
rt.tile([sl1, sl1_2], horz=False)

In [ ]:
sl2   =       spreadLines(node_focus='172.30.0.4', only_render_nodes=set(),                                  **params)
sl2_2 = rtsvg.spreadLines(node_focus='172.30.0.4', only_render_nodes=set(),                                  **params)
rt.tile([sl2, sl2_2], horz=False)

In [ ]:
# completes in 3s
_every_          = '1d'
df               = df.sort('timestamp')
s                = '172.30.0.4'
_df_             = df.group_by_dynamic('timestamp', every=_every_, group_by=['sip','dip']).agg()
_one_degree_     = _df_.filter((pl.col('sip') == s) | (pl.col('dip') == s))
_one_degree_set_ = set(_one_degree_['sip']) | set(_one_degree_['dip'])
_df_             = _df_.filter((pl.col('sip').is_in(_one_degree_set_)) | (pl.col('dip').is_in(_one_degree_set_)))
_df_             = _df_.sort('timestamp')
for k, k_df in _df_.group_by_dynamic('timestamp', every=_every_):
    ...